<a href="https://colab.research.google.com/github/AdriAIMeco/1-Repo/blob/main/TFC_Prediccion_Consumo_Energetico_en_un_Edificio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

# ------------------------
# 1. Creación o carga del dataset
# ------------------------
# En este ejemplo, generamos datos sintéticos de consumo energético.
# En un proyecto real, cargarías tu dataset real (por ejemplo, con pd.read_csv("tus_datos.csv")).

np.random.seed(42)
data_size = 1000

df = pd.DataFrame({
    'temp_in':    np.random.uniform(15, 25, data_size),   # Temperatura interior
    'temp_out':   np.random.uniform(-5, 35, data_size),   # Temperatura exterior
    'hum_in':     np.random.uniform(30, 60, data_size),   # Humedad interior
    'hum_out':    np.random.uniform(20, 80, data_size),   # Humedad exterior
    'hour':       np.random.randint(0, 24, data_size),    # Hora del día (0-23)
    'day_of_week':np.random.randint(0, 7, data_size),     # Día de la semana (0=Lunes,...,6=Domingo)
    'occupants':  np.random.randint(1, 5, data_size)      # Número de ocupantes
})

# Suponemos que el consumo energético es una función de las anteriores variables + ruido
df['energy_consumption'] = (
    df['temp_in'] * 0.5 +
    (35 - df['temp_out']) * 0.2 +
    df['hum_in'] * 0.3 +
    df['hum_out'] * 0.1 +
    df['hour'] * 0.7 +
    df['day_of_week'] * 0.8 +
    df['occupants'] * 5 +
    np.random.normal(0, 10, data_size)  # ruido
)

# ------------------------
# 2. Separación en features y target
# ------------------------
X = df.drop('energy_consumption', axis=1)
y = df['energy_consumption']

# ------------------------
# 3. División en train/test
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ------------------------
# 4. Entrenamiento de un modelo de regresión (RandomForest)
# ------------------------
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# (Opcional) Puedes imprimir la métrica en el conjunto de test
score = model.score(X_test, y_test)
print(f"R² en test: {score:.3f}")

# ------------------------
# 5. Guardar el modelo con joblib
# ------------------------
joblib.dump(model, "model.pkl")
print("Modelo entrenado y guardado en 'model.pkl'")
!pip install gradio

import gradio as gr
import joblib
import numpy as np

# Cargar el modelo previamente guardado
model = joblib.load("model.pkl")

def predict_energy(temp_in, temp_out, hum_in, hum_out, hour, day_of_week, occupants):
    """
    Función principal para predecir el consumo energético
    Dados los valores de variables ambientales y ocupación.
    """
    # Convertimos los inputs en un array 2D, tal como espera el modelo
    X = np.array([[temp_in, temp_out, hum_in, hum_out, hour, day_of_week, occupants]])

    # Realizar la predicción
    prediction = model.predict(X)

    # Devolver el valor (a 2 decimales)
    return f"{prediction[0]:.2f} kWh"

# Configurar la interfaz de Gradio
iface = gr.Interface(
    fn=predict_energy,
    inputs=[
        gr.Slider(minimum=0,  maximum=40, step=0.5, value=20, label="Temperatura Interior (°C)"),
        gr.Slider(minimum=0,  maximum=100, step=1,  value=50, label="Humedad Interior (%)"),
        gr.Slider(minimum=0,  maximum=100, step=1,  value=60, label="Humedad Exterior (%)"),
        gr.Slider(minimum=0,  maximum=23, step=1,  value=12, label="Hora del día"),
        gr.Slider(minimum=0,  maximum=6,  step=1,  value=2,  label="Día de la semana (0=Lunes,6=Dom)"),
        gr.Number(value=2,  label="Número de Ocupantes")
    ],
    outputs="text",
    title="Predicción de Consumo Energético (kWh)"
)

if __name__ == "__main__":
    iface.launch()





R² en test: 0.316
Modelo entrenado y guardado en 'model.pkl'


/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 7 arguments for function <function predict_energy at 0x78fe91d05300>, received 6.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 7 arguments for function <function predict_energy at 0x78fe91d05300>, received 6.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bafb782e452affae73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
